In [1]:
import scipy as sp, numpy as np, pandas as pd

## Discrete space

In [2]:
np.random.seed(42)
x = np.random.binomial(n=1, p=0.3, size=15) #n=trials, p=prob, size=number of samples
y = np.random.binomial(1,.7, 20)


clicks = np.array([ [np.sum(x), len(x)-np.sum(x)], [np.sum(y),len(y)-np.sum(y)] ]).T
pd.DataFrame(clicks, index=['Clicked', 'Not Clicked'], columns=['A','B'])

,A,B
Clicked,6,17
Not Clicked,9,3


In [3]:
print(x)

[0 1 1 0 0 0 0 1 0 1 0 1 1 0 0]


In [4]:
# OR: 
_, [a,b] = np.unique(x, return_counts=True)
_, [c,d] = np.unique(y, return_counts=True)
pd.DataFrame(data=[[b,d], [a,c]], # weird swapping required
                 index=["No click", "click"], 
                 columns=["A", "B"])

,A,B
No click,6,17
click,9,3


https://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/BS704_HypothesisTesting-ChiSquare/BS704_HypothesisTesting-ChiSquare_print.html

### Calculating P-value, ciritcal Value by "hand"
Example of a simple array -> see if the values fit a distribution

In [9]:
from scipy.stats import chisquare
print(chisquare([16, 18, 16, 14, 12, 12]))


exp = np.sum([16, 18, 16, 14, 12, 12]) / len([16, 18, 16, 14, 12, 12]) # expected => uniform unless stated differently
obs = np.array([16, 18, 16, 14, 12, 12])

print('Chi²: ', np.sum((obs-exp)**2 / exp))

# ppf p
critical_chi2 = sp.stats.chi2.ppf(1-0.05, len(obs)-1) # (conf-int, dof)
print('Critical Chi² value: ', critical_chi2)
print('P-value: ', 1 - sp.stats.chi2.cdf(2, len(obs)-1)) # p-value == 1-CDF( chi2, dof)

Power_divergenceResult(statistic=2.0, pvalue=0.8491450360846096)
Chi²:  2.0
11.070497693516351
P-value:  0.8491450360846096


#### Chi square test (pearson)

$h_0$: Independence of variables

$h_A$: No independence of variables

In [13]:
chi2, p_val, dof, expected = sp.stats.contingency.chi2_contingency(clicks) #, correction=False)
chi2, p_val, dof, expected

(5.835975241545894,
 0.015701703602172736,
 1,
 array([[ 9.85714286, 13.14285714],
        [ 5.14285714,  6.85714286]]))

In [14]:
# proof for expected (row*col / total):
## x11
print(np.sum(clicks[:,0]) * np.sum(clicks[0,:]) / np.sum(clicks), np.sum(clicks[:,1]) * np.sum(clicks[0,:]) / np.sum(clicks))
## x21
print(np.sum(clicks[:,0]) * np.sum(clicks[1,:]) / np.sum(clicks), np.sum(clicks[:,1]) * np.sum(clicks[1,:]) / np.sum(clicks))
## ...

9.857142857142858 13.142857142857142
5.142857142857143 6.857142857142857


In [15]:
# this is equivalent to above WHEN CORRECTION=False, otherwise 5.8359 !!!! (rounded:)
np.sum((clicks - np.array([[9.857,13.1429],[5.143, 6.857]]))**2 / np.array([[9.857,13.1429],[5.143, 6.857]]))
# == np.sum((clicks-expected)**2 / expected)

7.70327543166964

#### Fisher's exact test

Always look at cell matrix[0,0] as reference

In [16]:
pd.DataFrame(clicks, index=['Clicked', 'Not Clicked'], columns=['A','B'])

,A,B
Clicked,6,17
Not Clicked,9,3


In [17]:
odds_ratio, p_value = sp.stats.fisher_exact(clicks, alternative='two-sided')

print('odds_ratio: ', odds_ratio, '\np-value: ', p_value)

odds_ratio:  0.11764705882352941 
p-value:  0.010724879495736006


In [18]:
# odds ratio > 1?
#       true, false
# true  6     17
# false 9      3
## odds ratio: [ (true positives & true negatives) / (false postitives & false negatives) ]
(6*3) / (9*17)

0.11764705882352941

In [19]:
# hA greater = right tail = proportion of clicks in "A" is greater than the second (very unlikely given the table above)
sp.stats.fisher_exact(clicks, alternative='greater')

(0.11764705882352941, 0.9992829723658095)

In [20]:
# hA greater = right tail = proportion of clicks in "A" is smaller than the second (very likely given the table above)
sp.stats.fisher_exact(clicks, alternative='less')

(0.11764705882352941, 0.007554690396737117)

## Continuous space

#### For demonstration - Z-score examples
PPF -> percent point function == inverse of the CDF.

e.g. we want to know the (critical) Z value for 95%:

In [10]:
np.round(sp.stats.norm.ppf(.95),2)

1.64

In [11]:
np.round(sp.stats.norm.ppf(.975),2)

1.96

Before proceeding, make sure what parameters are "known":

Normal Distribution:
* known sigma²

T Distribution:
* unknown sigma²


1. Pooled/Weighted Sigma (equal variances)
2. Paired (Good to reduce noice, variances are equal, values are heterogenous)
3. Welch's Test for UNequal variances

In [12]:
# https://www.statology.org/working-with-the-student-t-distribution-in-r-dt-qt-pt-rt/
(1-sp.stats.t.cdf(x=.75, df=23))
sp.stats.t.ppf(.975, 200)

1.9718962236316089

In [21]:
np.random.seed(42)

x = sp.stats.norm(0,1)
y = sp.stats.uniform(loc=1,scale=2) # uniform between [1,1+2] -> 1 <= X <= 3

xi = x.rvs(10)
yi = y.rvs(10)

In [22]:
xi, yi

(array([ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986, -0.23415337,
        -0.23413696,  1.57921282,  0.76743473, -0.46947439,  0.54256004]),
 array([1.36364993, 1.36680902, 1.60848449, 2.04951286, 1.86389004,
        1.58245828, 2.22370579, 1.27898772, 1.5842893 , 1.73272369]))

two-sided -> 2*left value == left and right tail

$h_0$: both means are equal

$h_A$: means are different

In [26]:
sp.stats.ttest_ind(xi, yi, equal_var=True, alternative='two-sided')

Ttest_indResult(statistic=-4.9005554692298725, pvalue=0.00011519076433947554)

In [33]:
sp.stats.t.cdf(-4.9005554692298725, 18)*2

0.00011519076433947554

In [24]:
df = len(xi)+len(yi)-2
df

18

greater -> right tail. by default we get the left tail, so 1-left = right

$h_0$: xi mean <= yi group mean

$h_A$: xi mean > yi group mean

p-value ~ 1: cannot reject $h_0$, makes sense: xi is around zero, while yi is in [1,3] with mean ~ $\frac{1+3}{2}=2$

In [25]:
sp.stats.ttest_ind(xi, yi, equal_var=True, alternative='greater')

Ttest_indResult(statistic=-4.9005554692298725, pvalue=0.9999424046178302)

In [30]:
1-sp.stats.t.cdf(-4.9005554692298725, 18)

0.9999424046178302

less -> interested in left tail only

$h_0$: xi mean >= yi group mean

$h_A$: xi mean < yi group mean

p-value ~ 0: must reject $h_0$, makes sense: xi is around zero, while yi is in [1,3] with mean ~ $\frac{1+3}{2}=2$, so $h_A$: xi < yi is reasonable!

In [34]:
sp.stats.ttest_ind(xi, yi, equal_var=True, alternative='less')

Ttest_indResult(statistic=-4.9005554692298725, pvalue=5.759538216973777e-05)

In [35]:
sp.stats.t.cdf(-4.9005554692298725, 18)

5.759538216973777e-05

In [38]:
# test if xi is different to mu=1
sp.stats.ttest_1samp(xi, 1, alternative='less') # should be highly significant because:
# h0: xi >= 1
# hA: xi <  1
# We know that xi is centered around 0 !

Ttest_1sampResult(statistic=-2.4140579513473557, pvalue=0.01949435603492952)

reject $h_0$

In [51]:
s_x = 1
s_y = 2

z_score = np.mean(xi)-np.mean(yi) / np.sqrt( s_x**2/len(xi) + s_y**2/len(yi) )
z_score

normal_distr = sp.stats.norm(0,1)

2 * normal_distr.cdf(z_score)

0.05648920036699321